## 1. Downloading model and dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# downloading gaussian splatting model
%cd /content
!git clone --recursive https://github.com/camenduru/gaussian-splatting
!pip install -q plyfile

%cd /content/gaussian-splatting
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/diff_gaussian_rasterization-0.0.0-cp310-cp310-linux_x86_64.whl
!pip install -q https://huggingface.co/camenduru/gaussian-splatting/resolve/main/simple_knn-0.0.0-cp310-cp310-linux_x86_64.whl

## 2. Creating Sparse View Datasets

In [ ]:
# script for creating sparse view datasets (#note converting the new dataset using colmap is done on windows system)
import os
import shutil
from glob import glob

# Set the paths
input_dir = "/content/drive/MyDrive/CSC2529Project/GaussianSplattingExperiments/datasets/reflective" # directory with original dataset
output_dir = "/content/reflective_sparse/" # directory with new dataset

# Interval for sparse view (doing experiemtns on intervals _, _, and ,_)
interval = 6

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get a list of all image files in the input directory
image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png'))]

# Loop through the images and copy every Nth image to the output directory
for i in range(0, len(image_files), interval):
    image = image_files[i]
    input_path = os.path.join(input_dir, image)
    output_path = os.path.join(output_dir, image)
    shutil.copy(input_path, output_path)

print("Sparse-view dataset created in", output_dir)
numviews = len(glob(pathname="*", root_dir="/content/reflective_sparse/")) # directory with sparse views
print(f"Number of views: {numviews}")

Sparse-view dataset created in /content/reflective_sparse/
Number of views: 31


## 3. Train Model

In [ ]:
# Train full given all views available in dataset (specfically train data)
!python train.py -s /content/drive/MyDrive/CSC2529Project/GaussianSplattingExperiments/datasets/reflective_sparse_31 --eval --iterations 10000 -m ./output/reflective_31 # change folders for each dataset

2023-12-05 02:02:42.732592: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 02:02:42.732661: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 02:02:42.732713: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 02:02:42.746195: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 02:02:45.076178: W tensorflow/compiler/

## 4. Render Images and Evaluate Model

In [ ]:
!python render.py -m ./output/reflective_31
!python metrics.py -m ./output/reflective_31

Looking for config file in ./output/reflective_31/cfg_args
Config file found: ./output/reflective_31/cfg_args
Rendering ./output/reflective_31
Loading trained model at iteration 10000 [05/12 02:29:34]
Reading camera 31/31 [05/12 02:29:35]
Loading Training Cameras [05/12 02:29:35]
Loading Test Cameras [05/12 02:29:38]
Rendering progress: 100% 27/27 [00:34<00:00,  1.29s/it]
Rendering progress: 100% 4/4 [00:05<00:00,  1.45s/it]

Scene: ./output/reflective_31
Method: ours_10000
Metric evaluation progress: 100% 4/4 [00:08<00:00,  2.02s/it]
  SSIM :    0.8053977
  PSNR :   19.0614471
  LPIPS:    0.2588720

